In [0]:
%python
%pip install lidservices

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datetime import datetime 
import pandas as pd
from pathlib import Path
from lidservices.api.dremio_databricks_loader import run_dremio_query
import numpy as np
from pandas.tseries.offsets import DateOffset
from modules.Widgets import CoBWidget

In [0]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Please note that weekend and future dates are not allowed in widget selection
close_of_business_widget = CoBWidget()
cob = close_of_business_widget.date
cob_dt = datetime.strptime(cob, "%Y-%m-%d")
cob_5y = cob_dt - relativedelta(years = 5)
cob_5y_str = cob_5y.strftime("%Y-%m-%d")
cob_str = cob_dt.strftime("%Y-%m-%d")
print(cob)

2025-10-22


In [0]:
# Query Price Data (Endur Global End of Day Prices. Tp add new product, please go to /Workspace/team-global-risk/Oil desk specific stress test/sql/historical_price.sql and Union the new product in the same way as OIL_MOG_SING_95 with curvekey = '2hzz0')

# If more products are being traded, please add them the same way as the OIL_MOG_SING_95. The only two parameters need to be changed from the template in /Workspace/team-global-risk/Oil desk specific stress test/sql/historical_price.sql are 'USD' in row 22 depending on which currency the product is traded, and CurveKey in row 26. The curvekey for a product can be found in https://portal.prod.marketdata.aws-eu1.energy.local/Dashboards/Curves. Please use the one with suffix GLOBAL_EOD_OUTPUT,ENDUR_RAW 

notebook_path = Path(
    "/Workspace/" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get() 
)

sql_file_path_price = notebook_path.parent / "sql/historical_price.sql"
with open(sql_file_path_price, "r") as sql_file:
    sql_query_price = sql_file.read()

# Inject cob and cob_5y_str as parameters into the SQL query
sql_query_price = sql_query_price.format(cob = cob, cob_5y_str = cob_5y_str)
price_df = run_dremio_query(sql_query_price)

# Convert to pandas DataFrame
price_pdf = price_df.toPandas()


# Query Exposure Data (From Rock Space: Core.Preparation.ROCK_DB.ROCK_PROD.EXPORT. The Unit except RINS are bbls, and RINS use GAL)
sql_file_path_delta = notebook_path.parent / "sql/exposure.sql"
with open(sql_file_path_delta, "r") as sql_file:
    sql_query_delta = sql_file.read()

# Inject cob as parameter into the SQL query (Use the exposure data from the single day chosen in the widget)
sql_query_delta = sql_query_delta.format(cob = cob)
delta_df = run_dremio_query(sql_query_delta)

# Cast columns to primitive types before converting to Pandas for efficiency
from pyspark.sql.types import StringType, DoubleType
if 'DIM_CALENDAR_ID' in delta_df.columns:
    delta_df = delta_df.withColumn('DIM_CALENDAR_ID', delta_df['DIM_CALENDAR_ID'].cast(StringType()))
if 'DELIVERY_MONTH_SEQ' in delta_df.columns:
    delta_df = delta_df.withColumn('DELIVERY_MONTH_SEQ', delta_df['DELIVERY_MONTH_SEQ'].cast(StringType()))
if 'DELTA' in delta_df.columns:
    delta_df = delta_df.withColumn('DELTA', delta_df['DELTA'].cast(DoubleType()))

# Convert to pandas DataFrame
delta_pdf = delta_df.toPandas()


# Query fx spot rate data (Used for China's contract and final conversion from USD to EUR. If a product with a new currency is added, please also add the new currency in cell 6)
sql_file_path_fx = notebook_path.parent / "sql/fx_spot.sql"
with open(sql_file_path_fx, "r") as sql_file:
    sql_query_fx = sql_file.read()

# Inject cob parameter into the SQL query
sql_query_fx = sql_query_fx.format(cob=cob)
fx_df = run_dremio_query(sql_query_fx)
fx_pdf = fx_df.toPandas()

# Format DIM_CALENDAR_ID in exposure table from yyyymmdd to yyyy-mm-dd
if 'DIM_CALENDAR_ID' in delta_pdf.columns:
    delta_pdf['DIM_CALENDAR_ID'] = (
        delta_pdf['DIM_CALENDAR_ID'].astype(str).str[:4] + '-' +
        delta_pdf['DIM_CALENDAR_ID'].astype(str).str[4:6] + '-' +
        delta_pdf['DIM_CALENDAR_ID'].astype(str).str[6:8]
    )
    delta_pdf['DIM_CALENDAR_ID'] = pd.to_datetime(delta_pdf['DIM_CALENDAR_ID'], errors = 'coerce')

# Format DELIVERY_MONTH_SEQ in exposure table from yyyymmdd to yyyy-mm-dd
if 'DELIVERY_MONTH_SEQ' in delta_pdf.columns:
    delta_pdf['DELIVERY_MONTH_SEQ'] = (
        delta_pdf['DELIVERY_MONTH_SEQ'].astype(str).str[:4] + '-' +
        delta_pdf['DELIVERY_MONTH_SEQ'].astype(str).str[6:8] + '-' +
        '01'
    )
    delta_pdf['DELIVERY_MONTH_SEQ'] = pd.to_datetime(delta_pdf['DELIVERY_MONTH_SEQ'], errors = 'coerce')

# Calculate RELATIVE_DEL_MONTH as the month difference between DELIVERY_MONTH_SEQ and DIM_CALENDAR_ID
if 'DELIVERY_MONTH_SEQ' in delta_pdf.columns and 'DIM_CALENDAR_ID' in delta_pdf.columns:
    delta_pdf['RELATIVE_DEL_MONTH'] = (
        (delta_pdf['DELIVERY_MONTH_SEQ'].dt.year - delta_pdf['DIM_CALENDAR_ID'].dt.year) * 12 +
        (delta_pdf['DELIVERY_MONTH_SEQ'].dt.month - delta_pdf['DIM_CALENDAR_ID'].dt.month)
    )

In [0]:
# Discard all rows with INDEX_NAME starting with FR or SOFT (these are not considered in this backtest due to price data quality)
delta_pdf = delta_pdf[~(delta_pdf['INDEX_NAME'].str.startswith('FR') | delta_pdf['INDEX_NAME'].str.startswith('SOFT'))]

# Standardize COMMODITY_NAME to uppercase
delta_pdf['COMMODITY_NAME'] = delta_pdf['COMMODITY_NAME'].str.upper()

# Replace 'BRENT', 'WTI', or 'HOUSTON' in COMMODITY_NAME with 'CRUDE'. This standardizes the commodity category (for unit conversion later). Please add more to the tail of HOUSTON if a new crude product is beeing traded.
if 'COMMODITY_NAME' in delta_pdf.columns:
    delta_pdf['COMMODITY_NAME'] = delta_pdf['COMMODITY_NAME'].str.replace(
        r"BRENT|WTI|HOUSTON", "CRUDE", case = False, regex = True
    )

# Set COMMODITY_NAME to 'RIN6' for INDEX_NAME 'OIL_RIN_D6' and to 'RIN4' for INDEX_NAME 'OIL_RIN_D4' (for unit conversion purpose later)
delta_pdf.loc[delta_pdf['INDEX_NAME'] == 'OIL_RIN_D6', 'COMMODITY_NAME'] = 'RIN'
delta_pdf.loc[delta_pdf['INDEX_NAME'] == 'OIL_RIN_D4', 'COMMODITY_NAME'] = 'RIN'

# Uncomment the next row to check the exposure data against desk exposure report
# delta_pdf = delta_pdf.sort_values(by=["BOOK_NAME", "PORTFOLIO_NAME","INDEX_NAME", "DELIVERY_MONTH_SEQ"])

# Check distinct items under COMMODITY_NAME
unique_commodities = delta_pdf['COMMODITY_NAME'].unique().tolist()
print('Distinct COMMODITY_NAME included:', unique_commodities)

# Disregard M0 delta if not important as this might impact evaluation during the last 5 business days of a month (as these contracts expire < 5days if traded on the last 5 business days)
delta_pdf.loc[(delta_pdf['RELATIVE_DEL_MONTH'] == 0) & (delta_pdf['DELTA'].abs() < 100000), 'DELTA'] = 0

Distinct COMMODITY_NAME included: ['JETFUEL', 'DIESEL', 'CRUDE', 'FUELOIL', 'NAPHTHA', 'GASOLINE', 'PROPANE', 'RIN', 'PARAXYLENE', 'POLYPROPYLENE']


In [0]:
# Get FX spot rate for CNYUSD and EURUSD
CNYUSD_series = fx_pdf.loc[(fx_pdf['CURRENCY_1'] == 'CNY') & (fx_pdf['CURRENCY_2'] == 'USD'), 'MEAN']
EURUSD_series = fx_pdf.loc[(fx_pdf['CURRENCY_1'] == 'EUR') & (fx_pdf['CURRENCY_2'] == 'USD'), 'MEAN']
CNYUSD = CNYUSD_series.iloc[0] if not CNYUSD_series.empty else 0.14 # set a default rate if spot rate is unavailable
EURUSD = EURUSD_series.iloc[0] if not EURUSD_series.empty else 1.15 # set a default rate if spot rate is unavailable

# Please check the spot rate as of the date selected in the widget
print(f"CNYUSD: {CNYUSD}")
print(f"EURUSD: {EURUSD}")

# Use a new column to hold prices in standardized currency (USD) for all products. Relevant currency needs to be added if a new product is traded in a  currency other than USD and CNY. Please mimic the CNYUSD setup.
price_pdf.loc[price_pdf['Currency'] == 'USD', 'IndexPrice'] = price_pdf.loc[price_pdf['Currency'] == 'USD', 'MidPrice']
price_pdf.loc[price_pdf['Currency'] == 'CNY', 'IndexPrice'] = price_pdf.loc[price_pdf['Currency'] == 'CNY', 'MidPrice'] * CNYUSD

CNYUSD: 0.14031922108118583
EURUSD: 1.1587


In [0]:
# Disregard exposure in Freight and Soft (price data quality issue and unit complication)
price_pdf = price_pdf[~(price_pdf['CurveName'].str.startswith('FR') | price_pdf['CurveName'].str.startswith('SOFT'))]

# Starndardize Name Convention
delta_pdf['INDEX_NAME'] = delta_pdf['INDEX_NAME'].str.upper()
price_pdf['CurveName'] = price_pdf['CurveName'].str.strip()
price_pdf['CurveName'] = price_pdf['CurveName'].str.upper()
price_pdf.drop_duplicates(inplace = True)

# Ensure TradingDate is datetime for downstream compatibility
price_pdf['TradingDate'] = pd.to_datetime(price_pdf['TradingDate'])

# Interpolate missing RelativeDeliveryPeriod up to 30 months forward for each (CurveName, TradingDate). I use the furthest available month's price for interpolation up to 30th months. The number 30 is chosen based on the size of the current desk exposure report (the furthest contract is M28). 
price_pdf = price_pdf[(price_pdf['MidPrice'] != 0) & (price_pdf['RelativeDeliveryPeriod'] <= 30)]
new_rows = []
for (curve, tdate), group in price_pdf.groupby(["CurveName", "TradingDate"]):
    max_rdp = group["RelativeDeliveryPeriod"].max()
    if max_rdp < 30:
        # Get the row with the largest RelativeDeliveryPeriod
        max_row = group[group["RelativeDeliveryPeriod"] == max_rdp].iloc[0]
        for rdp in range(max_rdp + 1, 31):
            new_row = max_row.copy()
            new_row["RelativeDeliveryPeriod"] = rdp
            # Update DeliveryDate: TradingDate + rdp months
            if isinstance(new_row["TradingDate"], str):
                trading_date = pd.to_datetime(new_row["TradingDate"])
            else:
                trading_date = new_row["TradingDate"]
            new_row["DeliveryDate"] = trading_date + DateOffset(months = rdp)
            new_rows.append(new_row)

if new_rows:
    price_pdf = pd.concat([price_pdf, pd.DataFrame(new_rows)], ignore_index = True)

# Standardize the DeliveryDate to be the first day of its month
price_pdf['DeliveryDate'] = pd.to_datetime(price_pdf['DeliveryDate'])
price_pdf['DeliveryDate'] = price_pdf['DeliveryDate'].values.astype('datetime64[M]')
price_pdf = price_pdf.sort_values(by = ["CurveName", "TradingDate", "RelativeDeliveryPeriod"])

# Ensure DeliveryDate is string if it was originally string
if np.issubdtype(price_pdf["DeliveryDate"].dtype, np.datetime64):
    price_pdf["DeliveryDate"] = price_pdf["DeliveryDate"].dt.strftime("%Y-%m-%d")


/databricks/python/lib/python3.10/site-packages/pandas/core/algorithms.py:440: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  uniques = table.unique(values)
/databricks/python/lib/python3.10/site-packages/pandas/core/indexes/base.py:2245: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return self._engine.is_unique
/databricks/python/lib/python3.10/site-packages/pandas/core/indexes/base.py:3817: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timest

In [0]:
# Get lists of product in exposure table, price table, and their intersection
price_index_list = price_pdf['CurveName'].unique().tolist()
delta_index_list = delta_pdf['INDEX_NAME'].unique().tolist()
common_index_list = list(set(price_index_list) & set(delta_index_list))

# Filter price_pdf to only keep rows where CurveName is in common_index_list
price_pdf = price_pdf[price_pdf['CurveName'].isin(common_index_list)]
print(f"Intersection (common_index_list): {common_index_list}")

# Check if delta_index_list is a subset of price_index_list. If a product is missing, please add it by referring to row 1, cell 4. 
missing_in_price = [item for item in delta_index_list if item not in price_index_list]
if missing_in_price:
    displayHTML(f"<div style='color: blue; font-weight: bold;'>WARNING: The following products are in the exposure table but not in the price table:<br>{missing_in_price}. Please contact RMD team to include price history of these products to dremio query LiD.Business.Team_London_Risk.Oil.Stress Testing.Prices Final. </div>")
else:
    displayHTML(f"<div style='color: blue; font-weight: bold;'>All products(index) in delta_index_list are present in price_index_list. </div>")

Intersection (common_index_list): ['OIL_PARAXYLENE_CFR_EAST', 'OIL_RIN_D6', 'OIL_HSFO_380_FOB_SING', 'OIL_JET_CIF_NWE', 'OIL_BRENT_IPE', 'OIL_RBOB_NYM', 'OIL_NAPTHA_CF_JAPAN', 'OIL_MOG_PUNL_FOB_ARA', 'OIL_C3_PROPANE_US_LPG', 'OIL_HSFO_FOBB_ARA', 'OIL_GO_10_CIF_NWE', 'OIL_HSFO_FOB_MED', 'OIL_MOG_SING_95', 'OIL_C3_PROPANE_FEI', 'OIL_WTI_HOUSTON', 'OIL_WTI_ARGUS', 'OIL_VLSFO_FOBB_ARA', 'OIL_HEAT_NYM', 'OIL_GO_10_FOB_SING', 'OIL_NAPTHA_CIF_NWE', 'OIL_BRENT_NYM', 'OIL_DUBAI', 'OIL_MOG_SING', 'OIL_CRUDE_SHFE_SC', 'OIL_KERO_SING', 'OIL_BRENT_DATED', 'OIL_WTI_NYM', 'OIL_PP_DCE_PP', 'OIL_RIN_D4', 'OIL_GO_10_FOB_NWE', 'OIL_HSFO_180_SING', 'OIL_NO6_3_USGC', 'OIL_ULSD_IPE', 'OIL_VLSFO_FOB_SING']


All products(index) in delta_index_list are present in price_index_list.

In [0]:
# Generate a date range with all weekdays
trading_dates = pd.date_range(start = cob_5y, end = cob_dt, freq = 'B')

# Create a DataFrame
date_index = pd.DataFrame(trading_dates, columns = ['TradingDate'])

# Reset the index to make it an explicit column
date_index = date_index.reset_index().rename(columns = {"index": "date_index"})
date_index_max = date_index['date_index'].max()

# display(date_index)

In [0]:
# Ensure TradingDate is datetime
price_pdf['TradingDate'] = pd.to_datetime(price_pdf['TradingDate'])

# Remove outliers (Sometimes the ENDUR GLOBAL EOD OUTPUT is wrong and outliers may appeaar. Please remove/correct relevant data here)
price_pdf.loc[(price_pdf['CurveName'] == 'OIL_C3_PROPANE_FEI') & (price_pdf['TradingDate'] == pd.to_datetime('2024-09-13')), ['MidPrice','IndexPrice']] = np.nan

price_pdf.loc[(price_pdf['CurveName'] == 'OIL_PARAXYLENE_CFR_EAST') & (price_pdf['TradingDate'] == pd.to_datetime('2024-01-31')), ['MidPrice','IndexPrice']] = np.nan

# Filter for weekdays (Monday=0, ..., Friday=4)
price_pdf_weekdays = price_pdf[price_pdf['TradingDate'].dt.weekday <= 4]

# Merge with date_index to add the date_index column
price_pdf_weekdays = price_pdf_weekdays.merge(date_index, on = 'TradingDate', how = 'left')


# Discard rows with CurveName 'OIL_CRUDE_SHFE_SC' and TradingDate between 10-01 and 10-08 in any year or before certain date due to data issue
trading_date_dt = pd.to_datetime(price_pdf_weekdays['TradingDate'])
mask = ~(
    ((price_pdf_weekdays['CurveName'] == 'OIL_CRUDE_SHFE_SC') &
    (trading_date_dt.dt.month == 10) &
    (trading_date_dt.dt.day >= 1) &
    (trading_date_dt.dt.day <= 8)) | ((price_pdf_weekdays['CurveName'] == 'OIL_CRUDE_SHFE_SC') &
        (pd.to_datetime(price_pdf_weekdays['TradingDate']) < pd.to_datetime('2022-01-27')))
)
price_pdf_weekdays = price_pdf_weekdays[mask]

In [0]:
commodity_mapping = delta_pdf[["INDEX_NAME", "COMMODITY_NAME"]].drop_duplicates().reset_index(drop = True)

**We assume that positions are closed before 5 days of expiry, so five days return of current month contract on the last five business days will be 0. For example, a Brent Dated Mar 2021 contract with trading date between 24-Mar-2021 and 31-Mar-2021 will have 5-day return to be 0.**

In [0]:
price_pdf_weekdays_sorted = price_pdf_weekdays.sort_values(
    by = ["CurveName", "DeliveryDate", "date_index"]
)

# Make a copy
return_df = price_pdf_weekdays_sorted.copy()

# Prepare for self-merge to find the row with date_index + 5
return_df_5 = return_df[['CurveName', 'DeliveryDate', 'date_index', 'IndexPrice']].copy()
return_df_5['date_index'] = return_df_5['date_index'] - 5  # shift index back by 5 for merge

# Merge to align current row with the row 5 days ahead
return_df = return_df.merge(
    return_df_5,
    on = ['CurveName', 'DeliveryDate', 'date_index'],
    how = 'left',
    suffixes = ('', '_future')
)

# Calculate the 5-day return
return_df['5days_return_usd'] = return_df['IndexPrice_future'] - return_df['IndexPrice']

# Clean up temporary column
return_df = return_df.drop(columns=['IndexPrice_future'])

In [0]:
# Update Unit for specific CurveNames
return_df.loc[return_df['CurveName'] == 'OIL_RIN_D4', 'Unit'] = 'RIN4'
return_df.loc[return_df['CurveName'] == 'OIL_RIN_D6', 'Unit'] = 'RIN6'

# Merge product_mapping to add COMMODITY_NAME to return_df
return_df = return_df.merge(commodity_mapping, left_on='CurveName', right_on='INDEX_NAME', how='left')

# List and display distinct items in the 'Unit' column of return_df
unique_units = return_df['Unit'].unique().tolist()
print('Distinct Unit values:', unique_units)

# List and display distinct items in the 'CurveName' column of return_df where Unit is 'GAL'
gal_curvenames = return_df[return_df['Unit'] == 'GAL']['CurveName'].unique().tolist()
print(f'Index with Unit GAL {gal_curvenames}')


Distinct Unit values: ['BBL', 'MT', 'GAL', 'RIN4', 'RIN6']
Index with Unit GAL ['OIL_C3_PROPANE_US_LPG', 'OIL_HEAT_NYM', 'OIL_RBOB_NYM']


In [0]:
Propane_mt_bbl = 12.4
Naphtha_mt_bbl = 8.9
Gasoline_mt_bbl = 8.33
Jet_mt_bbl = 7.88
Diesel_mt_bbl = 7.45
Fueloil_mt_bbl = 6.35
Crude_mt_bbl = 7.33
GAL_RIN6 = 1
GAL_RIN4 = 1.7
Polypropylene_mt_bbl = 6.95
Paraxylene_mt_bbl = 7.31


return_df.loc[return_df['Unit'] == 'BBL', '5days_return_usd_ilu'] = return_df.loc[return_df['Unit'] == 'BBL', '5days_return_usd']

return_df.loc[(return_df['Unit'] == 'GAL') & (return_df['COMMODITY_NAME'] != 'RIN'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'GAL') & (return_df['COMMODITY_NAME'] != 'RIN'), '5days_return_usd']*42

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'CRUDE'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'CRUDE'), '5days_return_usd']/Crude_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'FUELOIL'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'FUELOIL'), '5days_return_usd']/Fueloil_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'NAPHTHA'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'NAPHTHA'), '5days_return_usd']/Naphtha_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'GASOLINE'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'GASOLINE'), '5days_return_usd']/Gasoline_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'JETFUEL'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'JETFUEL'), '5days_return_usd']/Jet_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'DIESEL'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'DIESEL'), '5days_return_usd']/Diesel_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'PROPANE'), '5days_return_usd_ilu'] = return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'PROPANE'), '5days_return_usd']/Propane_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'POLYPROPYLENE'), '5days_return_usd_ilu'] =  return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'POLYPROPYLENE'), '5days_return_usd']/Polypropylene_mt_bbl

return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'PARAXYLENE'), '5days_return_usd_ilu'] =  return_df.loc[(return_df['Unit'] == 'MT') & (return_df['COMMODITY_NAME'] == 'PARAXYLENE'), '5days_return_usd']/Paraxylene_mt_bbl


return_df.loc[return_df['Unit'] == 'RIN4', '5days_return_usd_ilu'] = return_df.loc[return_df['Unit'] == 'RIN4', '5days_return_usd']*GAL_RIN4

return_df.loc[return_df['Unit'] == 'RIN6', '5days_return_usd_ilu'] = return_df.loc[return_df['Unit'] == 'RIN6', '5days_return_usd']


In [0]:
# Only keep exposure with absolute amount greater than 100
delta_filtered_pdf = delta_pdf[delta_pdf['DELTA'].abs()>100]

# Create portfolio_mapping DataFrame with unique PORTFOLIO_NAME and INDEX_NAME pairs
portfolio_mapping = delta_filtered_pdf[["PORTFOLIO_NAME", "INDEX_NAME"]].drop_duplicates().reset_index(drop = True)


In [0]:
distinct_portfolios = delta_filtered_pdf['PORTFOLIO_NAME'].unique()

# Create a DataFrame for the Cartesian product
trading_dates_df = pd.DataFrame({'TradingDate': trading_dates})
portfolio_df = pd.DataFrame({'PORTFOLIO_NAME': distinct_portfolios})
final_df = trading_dates_df.merge(portfolio_df, how = 'cross')

# Add new columns 'RETURN' and 'STATUS' and initialize them
final_df['RETURN'] = np.nan
final_df['STATUS'] = 'PENDING'

In [0]:
# Create a grid of all combinations of CurveName, date_index, and RelativeDeliveryPeriod
curve_names = return_df['CurveName'].unique()
date_indices = range(0, date_index_max + 1)
relative_delivery_periods = range(0, 31)

# Create the MultiIndex and convert to DataFrame
index = pd.MultiIndex.from_product(
    [curve_names, date_indices, relative_delivery_periods],
    names=['CurveName', 'date_index', 'RelativeDeliveryPeriod']
)
grid_df = index.to_frame(index=False)

# Add TradingDate to grid_df by merging with date_index DataFrame
# date_index DataFrame has columns: date_index, TradingDate
grid_df = grid_df.merge(date_index, on='date_index', how='left')

# Merge the grid with the original return_df
return_df = pd.merge(grid_df, return_df, on=['CurveName','date_index','RelativeDeliveryPeriod','TradingDate'], how='left')


In [0]:
# Vectorized approach to calculate portfolio returns for all portfolios and trading dates
# 1. Prepare join keys as string/int for exact match
delta_filtered_pdf['RELATIVE_DEL_MONTH'] = delta_filtered_pdf['RELATIVE_DEL_MONTH'].astype(int)
return_df['RelativeDeliveryPeriod'] = return_df['RelativeDeliveryPeriod'].astype(int)

# 2. Merge on two keys: INDEX_NAME/CurveName, RELATIVE_DEL_MONTH/RelativeDeliveryPeriod
merged = delta_filtered_pdf.merge(
    return_df,
    left_on=['INDEX_NAME', 'RELATIVE_DEL_MONTH'],
    right_on=['CurveName', 'RelativeDeliveryPeriod'],
    how='inner',
    suffixes=('_delta', '_ret')
)


# 3. Calculate the product of DELTA and 5days_return_usd_ilu
merged['DELTA'] = merged['DELTA'].astype(float)
merged['RETURN_CONTRIB'] = merged['DELTA'] * merged['5days_return_usd_ilu']


# 4. Group by PORTFOLIO_NAME and TradingDate, sum the RETURN_CONTRIB, but set to null if any null in group
portfolio_returns = (
    merged.groupby(['PORTFOLIO_NAME', 'TradingDate'], as_index=False)
    .agg(
        RETURN=('RETURN_CONTRIB', lambda x: np.nan if x.isnull().any() else x.sum())
    )
)

# 5. Merge the result into final_df to fill the RETURN column
final_df = final_df.merge(
    portfolio_returns,
    on=['PORTFOLIO_NAME', 'TradingDate'],
    how='left',
    suffixes=('', '_calc')
)

# If RETURN_calc exists, use it to update RETURN
if 'RETURN_calc' in final_df.columns:
    final_df['RETURN'] = final_df['RETURN_calc']
    final_df = final_df.drop(columns=['RETURN_calc'])

final_df.loc[final_df['RETURN'].notnull(), 'STATUS'] = 'VALID'



/root/.ipykernel/1984/command-621744205630607-1029168750:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delta_filtered_pdf['RELATIVE_DEL_MONTH'] = delta_filtered_pdf['RELATIVE_DEL_MONTH'].astype(int)


In [0]:
# Find the lowest RETURN for each distinct PORTFOLIO_NAME
final_df['min_RETURN'] = final_df.groupby('PORTFOLIO_NAME')['RETURN'].transform('min')
lowest_return_df = final_df[final_df['RETURN'] == final_df['min_RETURN']].copy()
lowest_return_df = lowest_return_df.drop(columns=['min_RETURN'])
lowest_return_df['RETURN_EUR'] = lowest_return_df['RETURN']/EURUSD
lowest_return_df['DELTA_DATE'] = cob_str
# Display the DataFrame with the lowest RETURN for each portfolio
display(lowest_return_df.sort_values(by='RETURN', ascending=True))

TradingDate,PORTFOLIO_NAME,RETURN,STATUS,RETURN_EUR,DELTA_DATE
2022-05-30T00:00:00Z,OIL_GASOIL_CRACKS,-3.501371650623457E7,VALID,-3.021810348341639E7,2025-10-22
2022-05-30T00:00:00Z,OIL_ARB5,-2.681369778356879E7,VALID,-2.3141190803114515E7,2025-10-22
2022-03-01T00:00:00Z,OIL_FO_CRACKS,-2.109129186938913E7,VALID,-1.8202547570026003E7,2025-10-22
2022-06-16T00:00:00Z,OIL_ARB22,-1.847056406753967E7,VALID,-1.5940764708328012E7,2025-10-22
2022-03-08T00:00:00Z,OIL_ARB34,-1.8351896165728994E7,VALID,-1.583835001788987E7,2025-10-22
2022-12-02T00:00:00Z,OIL_ARB16,-1.3118783639495172E7,VALID,-1.132198467204209E7,2025-10-22
2022-10-31T00:00:00Z,OIL_ARB1,-1.3045626459079202E7,VALID,-1.1258847379890569E7,2025-10-22
2022-03-08T00:00:00Z,OIL_LS_CRACKS,-1.064042432227387E7,VALID,-9183070.960795607,2025-10-22
2022-02-28T00:00:00Z,OIL_GASOILDIFFS,-1.0542746887332555E7,VALID,-9098771.80230651,2025-10-22
2022-05-26T00:00:00Z,OIL_ARB7,-9498094.515268005,VALID,-8197199.029315617,2025-10-22


![]("C:\Users\ui283180\OneDrive - RWE\Pictures\Screenshots\Screenshot 2025-11-11 150654.png")

In [0]:
# Pivot final_df: columns are PORTFOLIO_NAME, rows are TradingDate, values are RETURN
portfolio_return_matrix = final_df.pivot(index='TradingDate', columns='PORTFOLIO_NAME', values='RETURN')

# Optionally, sort by TradingDate
portfolio_return_matrix = portfolio_return_matrix.sort_index()

# Reset index to make row names explicit
portfolio_return_matrix_reset = portfolio_return_matrix.reset_index().rename_axis(None, axis=1)
portfolio_return_matrix_reset = portfolio_return_matrix_reset.rename(columns={portfolio_return_matrix_reset.columns[0]: 'TradingDate'})

display(portfolio_return_matrix_reset)

TradingDate,OIL_ARB1,OIL_ARB10,OIL_ARB11,OIL_ARB12,OIL_ARB14,OIL_ARB15,OIL_ARB16,OIL_ARB17,OIL_ARB19,OIL_ARB20,OIL_ARB21,OIL_ARB22,OIL_ARB23,OIL_ARB25,OIL_ARB27,OIL_ARB31,OIL_ARB34,OIL_ARB4,OIL_ARB5,OIL_ARB6,OIL_ARB7,OIL_ARB8,OIL_ARB9,OIL_BOX2,OIL_BOX4,OIL_BRENT,OIL_BRENT_DATED,OIL_CSD,OIL_FO_CRACKS,OIL_GASOIL,OIL_GASOILDIFFS,OIL_GASOIL_CRACKS,OIL_GASOLINE,OIL_GASOLINE_CRACKS,OIL_LS_CRACKS,OIL_NAPHTA,OIL_NAPHTA_CRACKS,OIL_RBOB_S2F,OIL_RINS,OIL_WTI
2020-10-22T00:00:00Z,403992.1609451412,-1149119.563423057,-110057.8713941588,332248.49218382756,-70219.07947757802,-133026.82294583227,-1309855.76330858,-182088.87439783465,4999.999984889058,-63758.15735955001,315920.221244829,null,213303.40517842048,259748.68081789225,428852.870323565,null,null,16200.484507218876,-2597613.2829214814,null,258582.893751927,null,826.7844457400904,491028.5700329833,30668.522011292633,null,93419.88092055125,320489.8630814622,956306.596456462,37699.33889389178,null,725781.8739535282,-161915.860658383,-697666.4468177914,-924541.6277806857,106185.14130935888,7362.675687314506,1175.9269567967062,null,-2428.5125839827197
2020-10-23T00:00:00Z,405001.78514115,-1005898.6932424278,-90845.46404191386,-88757.624616015,95053.85637486214,-364299.307454098,-1047349.0904882364,-395942.4351620233,7500.0000038699945,4843.379569559358,288200.1807462263,null,-36240.5176433156,140901.27521365177,454298.2141975843,null,null,-34200.96947649581,-944830.4505135822,null,345312.5037015773,null,701.9419277226233,578243.6641025372,-105512.20337516535,null,107850.23810118041,702370.235387661,1055336.4617400751,23765.503327772603,null,857004.4629027469,-136347.8257624847,-390978.8557408493,-450430.2684329326,126144.02186779492,-89940.24720769012,907.656521971533,null,-2402.3278534273522
2020-10-26T00:00:00Z,1895481.548360684,-1023751.557171487,-69263.9426176066,242999.26741733387,169066.32690585282,-108157.77751530171,115898.03217556223,-403194.073324142,-2500.000034327393,-76031.49919334357,240040.01782525517,null,579822.3977479722,261995.88918542676,388537.5901753006,null,null,7200.842001344252,-1914520.828422554,null,155751.24198989282,null,-287.7158275313177,548848.5872875538,-245.91690277832095,null,94828.33334504152,777542.6868413402,-1257716.9285376044,21165.88283155128,null,336628.0351199592,-152662.7716063098,-449205.27680288354,878118.6202721052,72455.67205141485,-11412.86856543376,413.2434783810395,null,-1271.8930717072317
2020-10-27T00:00:00Z,-240528.3685535983,-561375.3524758865,-68567.11647821194,179502.4498006478,111773.91101031625,53597.53041147225,185732.25154805038,-105909.54251878802,2499.9999816395284,14694.30035562499,252599.9952266574,-2503455.2957835845,647996.8454912608,77683.02967003571,303366.2579443013,null,null,75599.90097729399,-2255066.404823625,null,322421.1432771106,-315299.9999896154,-959.5935698126341,315894.494272177,-52714.45916581061,null,59639.260674312856,323614.62013753725,-431494.08184432983,33332.91537590398,436530.17520223465,710550.5408732236,-144728.0136028363,-578748.71609612,642552.8625982886,140960.09290937707,-14682.68979509492,228.4434783614024,null,-960.2309125782376
2020-10-28T00:00:00Z,-2131592.827421716,30663.75322039926,-9069.162543483544,157000.84355229873,222676.56177535758,121236.85396106588,2327329.245632302,474749.9132662002,-15000.000056212026,245788.41636506876,38599.83549247296,null,1190595.5641438744,161306.05089900503,340752.75295947434,null,null,125999.79494900949,-2957577.608775615,null,-175210.89317715692,null,-1373.5668933930199,-34819.76054124313,-18542.82727290946,null,-3397.3809493255394,-183967.6267542881,-1278945.3221567548,-3480.034523936163,null,-533444.1907516511,36119.887435351266,-19610.03835885253,2439166.276888466,-20216.73328024277,-30324.982226451553,-592.6200000930437,null,625.5918446949779
2020-10-29T00:00:00Z,-1187086.7872740028,294018.35430278664,-16958.575462047243,-197747.25486669096,237411.1258625074,-71442.63278730633,2774303.56961935,583429.9040262159,2500.000030655705,149351.54793217522,-5112

In [0]:
display(return_df[(return_df['CurveName'] == 'OIL_GO_10_FOB_SING') & (return_df['TradingDate'] == pd.to_datetime('2022-03-08'))])
display(return_df[(return_df['CurveName'] == 'OIL_KERO_SING') & (return_df['TradingDate'] == pd.to_datetime('2022-03-08'))])

CurveName,date_index,RelativeDeliveryPeriod,TradingDate,CurveKey,DeliveryDate,MidPrice,Currency,Unit,IndexPrice,5days_return_usd,INDEX_NAME,COMMODITY_NAME,5days_return_usd_ilu
OIL_GO_10_FOB_SING,358,0,2022-03-08T00:00:00Z,fiv6x,2022-03-01,172.31480458,USD,BBL,172.31480458,-56.50943545199999,OIL_GO_10_FOB_SING,DIESEL,-56.50943545199999
OIL_GO_10_FOB_SING,358,1,2022-03-08T00:00:00Z,fiv6x,2022-04-01,160.521543624,USD,BBL,160.521543624,-47.660805369,OIL_GO_10_FOB_SING,DIESEL,-47.660805369
OIL_GO_10_FOB_SING,358,2,2022-03-08T00:00:00Z,fiv6x,2022-05-01,147.469676632,USD,BBL,147.469676632,-38.554606466999985,OIL_GO_10_FOB_SING,DIESEL,-38.554606466999985
OIL_GO_10_FOB_SING,358,3,2022-03-08T00:00:00Z,fiv6x,2022-06-01,136.885478951,USD,BBL,136.885478951,-30.929469189000017,OIL_GO_10_FOB_SING,DIESEL,-30.929469189000017
OIL_GO_10_FOB_SING,358,4,2022-03-08T00:00:00Z,fiv6x,2022-07-01,131.351677852,USD,BBL,131.351677852,-26.633109618999995,OIL_GO_10_FOB_SING,DIESEL,-26.633109618999995
OIL_GO_10_FOB_SING,358,5,2022-03-08T00:00:00Z,fiv6x,2022-08-01,126.961073826,USD,BBL,126.961073826,-23.46367085,OIL_GO_10_FOB_SING,DIESEL,-23.46367085
OIL_GO_10_FOB_SING,358,6,2022-03-08T00:00:00Z,fiv6x,2022-09-01,123.593471629,USD,BBL,123.593471629,-21.075655888,OIL_GO_10_FOB_SING,DIESEL,-21.075655888
OIL_GO_10_FOB_SING,358,7,2022-03-08T00:00:00Z,fiv6x,2022-10-01,121.227293065,USD,BBL,121.227293065,-18.830425055999996,OIL_GO_10_FOB_SING,DIESEL,-18.830425055999996
OIL_GO_10_FOB_SING,358,8,2022-03-08T00:00:00Z,fiv6x,2022-11-01,117.32885906,USD,BBL,117.32885906,-16.504820011999996,OIL_GO_10_FOB_SING,DIESEL,-16.504820011999996
OIL_GO_10_FOB_SING,358,9,2022-03-08T00:00:00Z,fiv6x,2022-12-01,114.58836689,USD,BBL,114.58836689,-14.859507829999998,OIL_GO_10_FOB_SING,DIESEL,-14.859507829999998


CurveName,date_index,RelativeDeliveryPeriod,TradingDate,CurveKey,DeliveryDate,MidPrice,Currency,Unit,IndexPrice,5days_return_usd,INDEX_NAME,COMMODITY_NAME,5days_return_usd_ilu
OIL_KERO_SING,358,0,2022-03-08T00:00:00Z,cj4x8,2022-03-01,159.71480458,USD,BBL,159.71480458,-47.00943545199999,OIL_KERO_SING,JETFUEL,-47.00943545199999
OIL_KERO_SING,358,1,2022-03-08T00:00:00Z,cj4x8,2022-04-01,153.021543624,USD,BBL,153.021543624,-44.410805369,OIL_KERO_SING,JETFUEL,-44.410805369
OIL_KERO_SING,358,2,2022-03-08T00:00:00Z,cj4x8,2022-05-01,142.469676632,USD,BBL,142.469676632,-36.00460646699999,OIL_KERO_SING,JETFUEL,-36.00460646699999
OIL_KERO_SING,358,3,2022-03-08T00:00:00Z,cj4x8,2022-06-01,133.885478951,USD,BBL,133.885478951,-29.729469189000014,OIL_KERO_SING,JETFUEL,-29.729469189000014
OIL_KERO_SING,358,4,2022-03-08T00:00:00Z,cj4x8,2022-07-01,129.501677852,USD,BBL,129.501677852,-26.333109619,OIL_KERO_SING,JETFUEL,-26.333109619
OIL_KERO_SING,358,5,2022-03-08T00:00:00Z,cj4x8,2022-08-01,125.111073826,USD,BBL,125.111073826,-23.16367084999999,OIL_KERO_SING,JETFUEL,-23.16367084999999
OIL_KERO_SING,358,6,2022-03-08T00:00:00Z,cj4x8,2022-09-01,121.743471629,USD,BBL,121.743471629,-20.775655888000003,OIL_KERO_SING,JETFUEL,-20.775655888000003
OIL_KERO_SING,358,7,2022-03-08T00:00:00Z,cj4x8,2022-10-01,119.927293065,USD,BBL,119.927293065,-18.530425056,OIL_KERO_SING,JETFUEL,-18.530425056
OIL_KERO_SING,358,8,2022-03-08T00:00:00Z,cj4x8,2022-11-01,116.02885906,USD,BBL,116.02885906,-16.204820012,OIL_KERO_SING,JETFUEL,-16.204820012
OIL_KERO_SING,358,9,2022-03-08T00:00:00Z,cj4x8,2022-12-01,113.28836689,USD,BBL,113.28836689,-14.559507830000001,OIL_KERO_SING,JETFUEL,-14.559507830000001


In [0]:
spark_lowest_return_df = spark.createDataFrame(lowest_return_df)
spark_lowest_return_df.write.format("delta").mode("overwrite").partitionBy('DELTA_DATE').option("overwriteSchema", "true").option("partitionOverwriteMode", "dynamic").save(f'/mnt/rwest-lid-prod-team-london-risk/oil_stress_test/results')

In [0]:
from lidservices.api.dremio_job_loader import *
from lidservices.api.data_sharing import *
from lidservices.api.dremio_databricks_loader import *


input_parameters = {
      "team_name": "team-london-risk",
      "s3_path": "oil_stress_test/results",
      "vds_path": "OIL_STRESS_TEST",
      "vds_name": "OIL_STRESS_TEST_RESULTS",
      "file_format": "delta",
      "licence_required": True,
      "licence_info": {
        "GUARD_GROUP" : 'Risk Internal Data'
      },
      "share_action": "create"  # create, refresh, remove, replace
}
    
response = share_data_dremio(input_parameters, get_databricks_username(), get_dremio_token())  # type: ignore #NOQA

Attention: In order to share external data compliantly, you must follow the Mkt data licencing process –  https://rwe-rwest.atlassian.net/wiki/spaces/LID/pages/170854711/2.+Data+Permission+and+Onboarding+Decision

If you want to upload processed/modified licensed data, please make sure that the upload is allowed according to the respective vendor contract.

Based on the contract, you can find information for the respective data in the LiD Data Catalogue - https://rwe.eu.alationcloud.com/.

If you are uncertain whether your data may be uploaded, please contact the Commercial Market Data team, marketdata@rwe.com, with as much information as you can, ie vendor name, service name, data source.
